In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [2]:
from combine import recombine
data = recombine("../data/potholes_1.csv", "../data/potholes_2.csv")

In [3]:
data["creation_date"] = pd.to_datetime(data["CREATION DATE"])
data["completion_date"] = pd.to_datetime(data["COMPLETION DATE"])
data["creation_year"] = pd.DatetimeIndex(data["creation_date"]).year

# Convert strings to times
data["creation_date"] = pd.to_datetime(data["creation_date"], format = "%m/%d/%Y")
data["completion_date"] = pd.to_datetime(data["completion_date"], format = "%m/%d/%Y")

# Create completion time variable
data["completion_time"] = data["completion_date"] - data["creation_date"]


In [4]:
data["completion_time"] = data["completion_time"].apply(lambda x: x.days)

In [5]:
data.drop('CREATION DATE', axis=1, inplace=True)
data.drop('COMPLETION DATE', axis=1, inplace=True)
data.drop('STATUS', axis=1, inplace=True)
data.drop('SERVICE REQUEST NUMBER', axis=1, inplace=True)
data.drop('TYPE OF SERVICE REQUEST', axis=1, inplace=True)
data.drop('CURRENT ACTIVITY', axis=1, inplace=True)
data.drop('MOST RECENT ACTION', axis=1, inplace=True)
data.drop('STREET ADDRESS', axis=1, inplace=True)
data.drop('X COORDINATE', axis=1, inplace=True)
data.drop('Y COORDINATE', axis=1, inplace=True)
data.drop('SSA', axis=1, inplace=True)
data.drop('LATITUDE', axis=1, inplace=True)
data.drop('LONGITUDE', axis=1, inplace=True)
data.drop('LOCATION', axis=1, inplace=True)
data.drop('creation_date', axis=1, inplace=True)
data.drop('completion_date', axis=1, inplace=True)
data.drop('ZIP', axis=1, inplace=True)
data.drop('Ward', axis=1, inplace=True)
data.drop('Police District', axis=1, inplace=True)
data.drop('NUMBER OF POTHOLES FILLED ON BLOCK', axis=1, inplace=True)
#data.drop('creation_year', axis=1, inplace=True)
data = data.dropna()
data["community_area"] = data["Community Area"].astype(int)
data.drop('Community Area', axis=1, inplace=True)
data.head


<bound method NDFrame.head of         creation_year  completion_time  community_area
0                2014            405.0               0
4                2018              4.0              24
5                2014             78.0              30
11               2014             19.0               0
13               2016              3.0              24
...               ...              ...             ...
560473           2014              0.0               3
560474           2011              0.0              32
560475           2015              0.0               8
560476           2014             31.0              77
560477           2015             75.0              24

[552844 rows x 3 columns]>

In [6]:
#data1 = data.groupby(["community_area"])["creation_year"].count()
#data1 = data1.drop(labels=0, axis=0)
#data1.head


data2 = data.groupby(["community_area"]).agg({'completion_time': 'mean'})
data2 = data2.drop(labels=0, axis=0)
data2.head

<bound method NDFrame.head of                 completion_time
community_area                 
1                     38.478614
2                     44.989105
3                     36.077008
4                     43.303720
5                     36.129124
...                         ...
73                    21.474498
74                    23.535413
75                    20.066487
76                    38.686128
77                    44.075829

[77 rows x 1 columns]>

In [7]:
crime = pd.read_csv("../data/Crimes_data.csv")

crime["community_area"] = crime["Community Area"]
crimes = crime[['community_area', "ID"]]
crimes.dropna()

crimec = crimes.groupby(["community_area"]).count()
crimec.head


<bound method NDFrame.head of                   ID
community_area      
1               3503
2               2347
3               3474
4               1343
5                746
...              ...
73              3104
74               374
75              1994
76              2198
77              1961

[77 rows x 1 columns]>

In [8]:
income = pd.read_csv("../data/chicago_census_data.csv")

income["income"] = income["PER CAPITA INCOME "]
income["community_area"] = income["Community Area Number"]
incomes = income[['community_area', "income"]]
incomes = incomes.drop(labels=77, axis=0)
incomes.head

<bound method NDFrame.head of     community_area  income
0              1.0   23939
1              2.0   23040
2              3.0   35787
3              4.0   37524
4              5.0   57123
..             ...     ...
72            73.0   19713
73            74.0   34381
74            75.0   27149
75            76.0   25828
76            77.0   33385

[77 rows x 2 columns]>